# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.127212e+02     1.519735e+00
 * time: 0.46071887016296387
     1     1.108686e+01     8.420007e-01
 * time: 1.5670578479766846
     2    -1.194859e+01     9.386606e-01
 * time: 1.6305267810821533
     3    -3.392449e+01     7.473458e-01
 * time: 1.7355108261108398
     4    -4.742102e+01     5.797032e-01
 * time: 1.8126518726348877
     5    -5.686962e+01     2.125174e-01
 * time: 1.900519847869873
     6    -5.974828e+01     1.280750e-01
 * time: 1.9777719974517822
     7    -6.080483e+01     4.868230e-02
 * time: 2.0300519466400146
     8    -6.122237e+01     8.362497e-02
 * time: 2.0826549530029297
     9    -6.154618e+01     3.499473e-02
 * time: 2.1355350017547607
    10    -6.178542e+01     2.938076e-02
 * time: 2.1891567707061768
    11    -6.196622e+01     2.207382e-02
 * time: 2.255751848220825
    12    -6.203889e+01     1.703372e-02
 * time: 2.308612823486328
    13    -6.211527e+01     1.413883e-02
 * time: 2.361400842666

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671069
    AtomicLocal         -18.8557727
    AtomicNonlocal      14.8522685
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485383 
    Xc                  -19.3336824

    total               -62.261666451544